In [ ]:
#1
from openai import OpenAI
from parser import code_to_file, extract_filename, run_code

In [ ]:
#2
client = OpenAI(
base_url = 'http://localhost:11434/v1',
api_key='ollama', # required, but unused
)

In [ ]:
#3
query = """Write a python program to check if a given number is Fibonacci number. Input:  8, 34, 41
"""
prompt = query + """
    Format the response in the following format:
    1. plan
    2. filename
    3. Code
    
    Do not show any comments. The code should not have multi line comments. Only single line comments are permissible.
    """

In [ ]:
#4
response = client.chat.completions.create(
model="qwen2.5:14b-instruct-q5_K_M",
messages=[
    {"role": "system", "content": "You are a helpful coding assistant"},
    {"role": "user", "content": prompt}
  ]
)

In [ ]:
#5
raw_response = response.choices[0].message.content
print(raw_response)    

In [ ]:
#6
response_refine = client.chat.completions.create(
model="qwen2.5:14b-instruct-q5_K_M",
messages=[
    {"role": "system", "content": "You are a helpful assistant. There are five conditions. Think carefully and follow the conditions strictly."},
    {"role": "user", "content": """You will analyze the text given, follow the instructions and provide the response. 
    Instructions:
    1. Enclose your response in following xml format tags: <code> and </code>. 
    2. Save the filename and code in dictionary format. keys will be 'filename' and 'code'.  {filename_key: filename_value, code_key: code_value}
    3. Within the dictionary, in the code key if you detect the python code use markdown blocks ```python in the beginning and ``` in the end. 
    4. Use escaped triple quotes (`\"\"\"`) for the value of the key instead of triple quotes.
    5. If you detect multi line comments in triple quotes within the value of the code, remove those comments. Use single line comments
    6. Ensure you have followed the correct openings and closings like <code> </code> tags and { } curly brackets
    
    Eample: 
<code>
{
  "filename": "hello.py",
  "code": ```python
  print("Hello world!")
  ```
}
</code>

Here is the text for you: """ + raw_response}
  ]
)

In [ ]:
#7
print(response_refine.choices[0].message.content)

In [ ]:
#8
x=response_refine.choices[0].message.content

In [ ]:
#9
code_to_file(x)

In [ ]:
#10
print(x)

In [ ]:
#11
file_name = extract_filename(x)
print(file_name)

In [ ]:
#12
code_outcome=run_code(file_name,'septimus')

#### Possible places where we get error:
1. Line 9 --> if this fails, start running from line 6
2. Line 11 (the extension may not be corret) --> if this fails, start running from line 6
3. Line 12 (The code is incorrect) --> if this fails, it should edit the file.

#### Next steps:
This entire script should be a wrapper. Think about the inputs.

In [ ]:
#13
print(code_outcome)

In [ ]:
#14
print(code_outcome[1])

### This needs another solid planning. Need to write a flowchart to iron out the exact flow and then proceed with the code